## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pickle
%matplotlib qt

## Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.

In [2]:
# Get objpoints, imgpoints
def calibrate_camera(images):
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.
    for idx,fname in enumerate(images):
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
            img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
            #cv2.imshow(fname,img)
            write_name = '../output_images/chessboard_corners/' + str(idx) + '.jpg' 
            #cv2.imwrite(write_name, img)      
            cv2.waitKey(100)

        cv2.destroyAllWindows()
    return objpoints, imgpoints

# Get camera calibration matrix and distortion coefficients
def distort_coeffs(img, objpoints, imgpoints):
    img = mpimg.imread(img)
    img_size = (img.shape[1], img.shape[0])
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)
    return mtx, dist

In [3]:
# Undistort image function
def cal_undistort(img, mtx, dist):
    undst = cv2.undistort(img, mtx, dist, None, mtx)
    return undst

# Test undistortion on a chessboard images
calibration_images = glob.glob('../camera_cal/calibration*.jpg')
objpoints, imgpoints = calibrate_camera(calibration_images)
chessboard = '../camera_cal/calibration1.jpg'
mtx, dist = distort_coeffs(chessboard, objpoints, imgpoints)

chessboard_images = glob.glob('../camera_cal/calibration*.jpg')

for idx,fname in enumerate(chessboard_images):
    chessboard_img = cv2.imread(fname)
    chessboard_undst = cal_undistort(chessboard_img, mtx, dist)
    write_name = '../output_images/undistorted_chessboards/' + str(idx) + '.jpg'
    #cv2.imwrite(write_name, chessboard_undst)      

    # Visualize undistortion
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    ax1.imshow(chessboard_img)
    ax1.set_title(fname, fontsize=12)
    ax2.imshow(chessboard_undst)
    ax2.set_title(write_name, fontsize=12)

## Apply a distortion correction to raw images.

test_images = glob.glob('../test_images/test*.jpg')
#test_images = glob.glob('../test_images/straight_lines*.jpg')

for idx,test_image in enumerate(test_images):
    dist_img = mpimg.imread(test_image)
    test_undst = cal_undistort(dist_img, mtx, dist)
    #write_name = '../output_images/undistorted_test_images/test' + str(idx) + '.jpg'
    #write_name = '../output_images/undistorted_test_images/straight_lines' + str(idx) + '.jpg'
    #cv2.imwrite(write_name, test_undst)      
    
    # Visualize undistortion
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    ax1.imshow(dist_img)
    ax1.set_title('Original Distorted Image', fontsize=20)
    ax2.imshow(test_undst)
    ax2.set_title('Undistorted Image', fontsize=20)

## Use color transforms, gradients, etc., to create a thresholded binary image.

### Pre-process images
### Undistort, Grayscale, derive Sobel operators

# Read in the saved camera matrix and distortion coefficients
dist_pickle = pickle.load( open( "../output_images/wide_dist_pickle.p", "rb" ) )
mtx = dist_pickle["mtx"]
dist = dist_pickle["dist"]

# Sobel kernel size
ksize = 3

In [4]:
def process_image(img, sobel_kernel=3):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    abs_sobelx = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel))
    abs_sobely = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel))
    return abs_sobelx, abs_sobely

## Gradients

In [5]:
# Scaled Sobel threshold function
def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    abs_sobelx, abs_sobely = process_image(img, sobel_kernel)
    if orient == 'x': abs_sobel = abs_sobelx
    if orient == 'y': abs_sobel = abs_sobely
    scaled_sobel = np.uint8(255 * abs_sobel / np.max(abs_sobel))
    grad_binary = np.zeros_like(scaled_sobel)
    grad_binary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    return grad_binary
    
#grad_binary = abs_sobel_thresh(sample_img, orient='x', sobel_kernel=3, thresh=(50,150))
#f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
#f.tight_layout()
#ax1.imshow(sample_img)
#ax1.set_title('Original Image', fontsize=30)
#ax2.imshow(grad_binary, cmap='gray')
#ax2.set_title('Thresholded Gradient', fontsize=30)
#plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [6]:
# Gradient magnitude threshold function
def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    abs_sobelx, abs_sobely = process_image(img, sobel_kernel)
    magnitude = np.sqrt(abs_sobelx**2 + abs_sobely**2)
    scaled_sobel = np.uint8(255 * magnitude / np.max(magnitude))
    mag_binary = np.zeros_like(scaled_sobel)
    mag_binary[(scaled_sobel >= mag_thresh[0]) & (scaled_sobel <= mag_thresh[1])] = 1
    return mag_binary
    
#mag_binary = mag_thresh(sample_img, sobel_kernel=9, mag_thresh=(40, 140))
#f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
#f.tight_layout()
#ax1.imshow(sample_img)
#ax1.set_title('Original Image', fontsize=20)
#ax2.imshow(mag_binary, cmap='gray')
#ax2.set_title('Thresholded Magnitude', fontsize=20)
#plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [7]:
# Gradient direction threshold function
def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    abs_sobelx, abs_sobely = process_image(img, sobel_kernel)
    direction = np.arctan2(abs_sobely, abs_sobelx)
    dir_binary = np.zeros_like(direction)
    dir_binary[(direction >= thresh[0]) & (direction <= thresh[1])] = 1
    return dir_binary
    
#dir_binary = dir_threshold(sample_img, sobel_kernel=15, thresh=(0.8, 1.2))
#f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
#f.tight_layout()
#ax1.imshow(sample_img)
#ax1.set_title('Original Image', fontsize=20)
#ax2.imshow(dir_binary, cmap='gray')
#ax2.set_title('Thresholded Grad. Dir.', fontsize=20)
#plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

## Colorspace

In [8]:
# Grayscale threshold function
def gray_threshold(img, thresh=(0, 255)):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray_binary = np.zeros_like(gray)
    gray_binary[(gray > thresh[0]) & (gray <= thresh[1])] = 1
    return gray_binary

#gray_binary = gray_threshold(sample_img, thresh = (205, 255))
#f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
#f.tight_layout()
#ax1.imshow(cv2.cvtColor(sample_img, cv2.COLOR_RGB2GRAY), cmap='gray')
#ax1.set_title('Gray Image', fontsize=20)
#ax2.imshow(gray_binary, cmap='gray')
#ax2.set_title('Thresholded Gray', fontsize=20)
#plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [9]:
# RGB channels threshold function
def rgb_threshold(img, channel='r', thresh=(0, 255)):
    R = img[:,:,0]
    G = img[:,:,1]
    B = img[:,:,2]
    if channel == 'r': ch = R
    if channel == 'g': ch = G
    if channel == 'b': ch = B
    rgb_binary = np.zeros_like(ch)
    rgb_binary[(ch > thresh[0]) & (ch <= thresh[1])] = 1
    return rgb_binary

#rgb_binary = rgb_threshold(sample_img, channel='r', thresh=(225, 255))
#f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
#f.tight_layout()
#ax1.imshow(sample_img)
#ax1.set_title('Original Image', fontsize=20)
#ax2.imshow(rgb_binary, cmap='gray')
#ax2.set_title('Thresholded Red', fontsize=20)
#plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [10]:
# HLS colorspace threshold function
def hls_threshold(img, channel='s', thresh=(0, 255)):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    H = hls[:,:,0]
    L = hls[:,:,1]
    S = hls[:,:,2]
    if channel == 'h': ch = H
    if channel == 'l': ch = L
    if channel == 's': ch = S
    binary = np.zeros_like(ch)
    binary[(ch > thresh[0]) & (ch <= thresh[1])] = 1
    return binary

#hls_binary = hls_threshold(sample_img, channel='s', thresh=(170, 255))
#f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
#f.tight_layout()
#ax1.imshow(sample_img)
#ax1.set_title('Original Image', fontsize=20)
#ax2.imshow(hls_binary, cmap='gray')
#ax2.set_title('Thresholded S Channel', fontsize=20)
#plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

### Apply and combine thresholds

In [27]:
def threshold_combo(img):
    img = np.copy(img) 

    # Threshold gray 
    gr_binary = gray_threshold(img, thresh = (205, 255))

    # Threshold S channel
    s_binary = hls_threshold(img, channel='s', thresh=(205, 255))

    # Threshold L channel
    l_binary = hls_threshold(img, channel='l', thresh=(200, 255))

    # Threshold R channel
    r_binary = rgb_threshold(img, channel='r', thresh=(225, 255))
    
    # Combined color threshold
    slr_binary = np.zeros_like(s_binary)
    slr_binary[ [(s_binary == 1) | (l_binary == 1)  | (r_binary == 1)] ] = 1

    # Threshold x gradient
    sxbinary = abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(20, 100))
    
    # Threshold color channel
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, slr_binary))

    # Combined binary thresholds
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(slr_binary == 1) | (sxbinary == 1) | (gr_binary == 1)] = 1

    return color_binary, combined_binary
    
#color_binary, combined_binary = threshold_combo(sample_img)
#f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
#f.tight_layout()
#ax1.imshow(color_binary)
#ax1.set_title('Color Binary', fontsize=20)
#ax2.imshow(combined_binary, cmap='gray')
#ax2.set_title('Combined Binary', fontsize=20)
#plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

## Apply a perspective transform to rectify binary image ("birds-eye view").

In [28]:
def get_shapes(img):
    img_size = (img.shape[1], img.shape[0])
    # left,top --> left,bottom --> right,bottom --> right, top
    top = img_size[1] * 0.62
    bottom = img_size[1] 
    left_top = img_size[0] * 0.47
    right_top = img_size[0] * 0.53
    left_bottom = img_size[0] * 0.155
    right_bottom = img_size[0] * 0.87
    src = np.float32 ([[left_top, top], [left_bottom,bottom], [right_bottom, bottom], [right_top, top]])
    dst = np.float32([[img_size[0] /4, 0], [img_size[0] /4, img_size[1]], [img_size[0] * 3/4, img_size[1] ], [img_size[0] * 3/4, 0]])
    return src, dst

In [29]:
def corners_unwarp(img, src, dst):
    img_size = (img.shape[1], img.shape[0])
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    Minv = cv2.getPerspectiveTransform(dst,src)
    return warped, M, Minv

# undst_sample_img, color_binary, combined_binary
#binary_warped, perspective_M, Minv = corners_unwarp(combined_binary, src, dst)
#warped, perspective_M, Minv = corners_unwarp(undst_sample_img, src, dst)
#f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
#f.tight_layout()
#ax1.imshow(undst_sample_img, cmap='gray')
#ax1.set_title('Undistorted Sample Image', fontsize=20)
#ax2.imshow(binary_warped, cmap='gray')
#ax2.set_title('Undistorted Warped Image', fontsize=20)
#plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [30]:
# check src coordinates
def draw_lines(img, src):
    img = np.copy(img)
    src = src.astype(np.int32)
    src = src.reshape((-1,1,2))
    img = cv2.polylines(img,[src], True, (255,255,255), 3)
    return img

#region = draw_lines(undist_sample_img, src)
#plt.imshow(region)
#src = src.astype(np.int32)
#print(src)

In [31]:
# check dst coordinates
def draw_lines(img, dst):
    img = np.copy(img)
    dst = dst.astype(np.int32)
    dst = dst.reshape((-1,1,2))
    img = cv2.polylines(img,[dst], True, (255,255,255), 5)
    return img

#region = draw_lines(warped, dst)
#plt.imshow(region)
#dst = dst.astype(np.int32)
#print(dst)

## Detect lane pixels and fit to find the lane boundary.

In [32]:
def detect_lane_boundary(binary_warped):
    # Histogram version for detecting lane pixels 
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    # Peak of the left and right halves of the histogram = starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    # Number of sliding windows
    nwindows = 9
    # Height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Width of the windows +/- margin
    margin = 100
    # Minimum number of pixels found to recenter window
    minpix = 50
    # Left and right lane pixel indices lists
    left_lane_inds = []
    right_lane_inds = []

    for window in range(nwindows):
        # Window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin

        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 

        # Nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]

        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # Recenter next window on their mean position if > minpix pixels
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Second order polynomial of each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Visualization
    # x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    # Raw data
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    #plt.imshow(out_img)

    # Plot lines
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    #plt.xlim(0, 1280)
    #plt.ylim(720, 0)
    
    # Lane curvature
    left_curverad, right_curverad, mean_curverad = lane_curvature(ploty, leftx, lefty, rightx, righty)

    return left_fit, right_fit, left_fitx, right_fitx, ploty, left_curverad, right_curverad, mean_curverad

#left_fit, right_fit, left_fitx, right_fitx, ploty, left_curverad, right_curverad, mean_curverad = detect_lane_boundary(binary_warped)

In [33]:
# FOR VIDEO
# binary_warped = combined_binary for new image
# Nonzero pixels in x and y within new window
def detect_lane_boundary2(binary_warped, left_fit, right_fit):
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100

    # Use the polynomial derived from the old image 
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))  

    # Left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Second order polynomial for left and right line pixel positions
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    # out_img = an image to draw on 
    # window_img = an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)

    # Left and right line pixels in out_img
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    #plt.imshow(result)
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    #plt.xlim(0, 1280)
    #plt.ylim(720, 0)
    
    # Lane curvature
    left_curverad, right_curverad, mean_curverad = lane_curvature(ploty, leftx, lefty, rightx, righty)

    return left_fit, right_fit, left_fitx, right_fitx, ploty, left_curverad, right_curverad, mean_curverad

#left_fit, right_fit, left_fitx, right_fitx, ploty, left_curverad, right_curverad, mean_curverad = detect_lane_boundary2(binary_warped, left_fit, right_fit)


## Determine the curvature of the lane and vehicle position with respect to center.

In [34]:
def lane_curvature(ploty, leftx, lefty, rightx, righty):
    y_eval = np.max(ploty)
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)

    # New radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    mean_curverad = round(np.mean([left_curverad, right_curverad]), 2)
    return left_curverad, right_curverad, mean_curverad


## Warp the detected lane boundaries back onto the original image.

In [35]:
# warped = binary_warped of the undst_sample_img
def reverse_transform(undst_sample_img, warped, left_fitx, right_fitx, ploty, Minv):
    # warp_zero = an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    #color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    color_warp = warp_zero

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (warped.shape[1], warped.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(undst_sample_img, 1, newwarp, 0.3, 0)
    #plt.imshow(result)
    return result

#reverse_transform(undst_sample_img, warped, left_fitx, right_fitx, ploty, Minv)


## Determine the curvature of the lane and vehicle position with respect to center.

In [36]:
def lane_curvature(ploty, leftx, lefty, rightx, righty):
    y_eval = np.max(ploty)
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)

    # New radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    mean_curverad = round(np.mean([left_curverad, right_curverad]), 2)
    return left_curverad, right_curverad, mean_curverad


## Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

In [37]:
# Compute offset
def show_lane(result, left_fitx, right_fitx, mean_curverad):
    img_ctr = result.shape[1] / 2
    lane_ctr = ((right_fitx[-1] - left_fitx[-1]) / 2) + left_fitx[-1]
    offset = round((img_ctr - lane_ctr) * 3.7 / 700, 2)

    text1 = 'estimation of lane curvature: ' + str(mean_curverad) + ' m'
    org1 = (50,50)
    text2 = 'vehicle position from center : ' + str(offset) + ' m'
    org2 = (50,100)
    font = cv2.FONT_HERSHEY_SIMPLEX
    scale = 1.5
    color = (255,255,255)
    thickness = 2
    cv2.putText(result, text1, org1, font, scale, color, thickness)
    cv2.putText(result, text2, org2, font, scale, color, thickness)
    return result

#result = show_lane(result, left_fitx, right_fitx, mean_curverad)
#plt.imshow(result)

## Main Pipeline function

In [38]:
def pipeline(img, s_thresh=(170, 255), sx_thresh=(20, 100)):
    #img = mpimg.imread(img)
    img = np.copy(img)
    # Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
    # Read in the saved camera matrix and distortion coefficients
    dist_pickle = pickle.load( open( "../output_images/wide_dist_pickle.p", "rb" ) )
    mtx = dist_pickle["mtx"]
    dist = dist_pickle["dist"]
    # Apply a distortion correction to raw images.
    undst = cal_undistort(img, mtx, dist)
    # Use color transforms, gradients, etc., to create a thresholded binary image.
    color_binary, combined_binary = threshold_combo(undst)
    #Apply a perspective transform to rectify binary image ("birds-eye view").
    src, dst = get_shapes(undst)
    binary_warped, perspective_M, Minv = corners_unwarp(combined_binary, src, dst)
    #Detect lane pixels and fit to find the lane boundary.
    #Determine the curvature of the lane and vehicle position with respect to center.
    #left_fit = np.array([0,0,0])
    #right_fit = np.array([0,0,0])
    left_fit = np.empty([3])
    right_fit = np.empty([3])
    #if not (left_fit.size & right_fit.size):
    left_fit, right_fit, left_fitx, right_fitx, ploty, left_curverad, right_curverad, mean_curverad = detect_lane_boundary(binary_warped)
    #else:
    #    left_fit, right_fit, left_fitx, right_fitx, ploty, left_curverad, right_curverad, mean_curverad = detect_lane_boundary2(binary_warped, left_fit, right_fit)
    #Warp the detected lane boundaries back onto the original image.
    warped, perspective_M, Minv = corners_unwarp(undst, src, dst)
    unwarped = reverse_transform(undst, warped, left_fitx, right_fitx, ploty, Minv)
    #Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.
    result = show_lane(unwarped, left_fitx, right_fitx, mean_curverad)
    return result

## Test on images

In [ ]:
# Calibrate camera
calibration_images = glob.glob('../camera_cal/calibration*.jpg')
objpoints, imgpoints = calibrate_camera(calibration_images)
chessboard = '../camera_cal/calibration1.jpg'
mtx, dist = distort_coeffs(chessboard, objpoints, imgpoints)

dist_pickle = {}
dist_pickle["mtx"] = mtx
dist_pickle["dist"] = dist
pickle.dump( dist_pickle, open( "../output_images/wide_dist_pickle.p", "wb" ) )

In [39]:
# List of test images
#test_images = glob.glob('../test_images/straight_lines*.jpg')
test_images = glob.glob('../test_images/test*.jpg')

# Test pipeline on images
for idx, test_image in enumerate(test_images):
    img_name = test_image
    test_img = mpimg.imread(img_name)
    test_result = pipeline(test_img)
    
    # Plot the result
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(test_img)
    ax1.set_title('Original Image', fontsize=30)
    ax2.imshow(test_result)
    ax2.set_title('Pipeline Result', fontsize=30)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

    # Save images to output folder
    #write_name = '../output_images/result_test_images/' + str(idx) + '.jpg'
    #cv2.imwrite(write_name, test_result)      

In [ ]:
# Convolution version
# Read in a thresholded image
warped = binary_warped
# window settings
window_width = 50 
window_height = 80 # Break image into 9 vertical layers since image height is 720
margin = 100 # How much to slide left and right for searching

def window_mask(width, height, img_ref, center, level):
    output = np.zeros_like(img_ref)
    output[int(img_ref.shape[0]-(level+1)*height):int(img_ref.shape[0]-level*height),max(0,int(center-width/2)):min(int(center+width/2),img_ref.shape[1])] = 1
    return output

def find_window_centroids(image, window_width, window_height, margin):
    
    window_centroids = [] # Store the (left,right) window centroid positions per level
    window = np.ones(window_width) # Create our window template that we will use for convolutions
    
    # First find the two starting positions for the left and right lane by using np.sum to get the vertical image slice
    # and then np.convolve the vertical image slice with the window template 
    
    # Sum quarter bottom of image to get slice, could use a different ratio
    l_sum = np.sum(warped[int(3*warped.shape[0]/4):,:int(warped.shape[1]/2)], axis=0)
    l_center = np.argmax(np.convolve(window,l_sum))-window_width/2
    r_sum = np.sum(warped[int(3*warped.shape[0]/4):,int(warped.shape[1]/2):], axis=0)
    r_center = np.argmax(np.convolve(window,r_sum))-window_width/2+int(warped.shape[1]/2)
    
    # Add what we found for the first layer
    window_centroids.append((l_center,r_center))
    
    # Go through each layer looking for max pixel locations
    for level in range(1,(int)(warped.shape[0]/window_height)):
        # convolve the window into the vertical slice of the image
        image_layer = np.sum(warped[int(warped.shape[0]-(level+1)*window_height):int(warped.shape[0]-level*window_height),:], axis=0)
        conv_signal = np.convolve(window, image_layer)
        # Find the best left centroid by using past left center as a reference
        # Use window_width/2 as offset because convolution signal reference is at right side of window, not center of window
        offset = window_width/2
        l_min_index = int(max(l_center+offset-margin,0))
        l_max_index = int(min(l_center+offset+margin,warped.shape[1]))
        l_center = np.argmax(conv_signal[l_min_index:l_max_index])+l_min_index-offset
        # Find the best right centroid by using past right center as a reference
        r_min_index = int(max(r_center+offset-margin,0))
        r_max_index = int(min(r_center+offset+margin,warped.shape[1]))
        r_center = np.argmax(conv_signal[r_min_index:r_max_index])+r_min_index-offset
        # Add what we found for that layer
        window_centroids.append((l_center,r_center))

    return window_centroids

window_centroids = find_window_centroids(warped, window_width, window_height, margin)

# If we found any window centers
if len(window_centroids) > 0:

    # Points used to draw all the left and right windows
    l_points = np.zeros_like(warped)
    r_points = np.zeros_like(warped)

    # Go through each level and draw the windows 	
    for level in range(0,len(window_centroids)):
        # Window_mask is a function to draw window areas
        l_mask = window_mask(window_width,window_height,warped,window_centroids[level][0],level)
        r_mask = window_mask(window_width,window_height,warped,window_centroids[level][1],level)
        # Add graphic points from window mask here to total pixels found 
        l_points[(l_points == 255) | ((l_mask == 1) ) ] = 255
        r_points[(r_points == 255) | ((r_mask == 1) ) ] = 255

    # Draw the results
    template = np.array(r_points+l_points,np.uint8) # add both left and right window pixels together
    zero_channel = np.zeros_like(template) # create a zero color channel
    template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8) # make window pixels green
    warpage = np.array(cv2.merge((warped,warped,warped)),np.uint8) # making the original road pixels 3 color channels
    output = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) # overlay the orignal road image with window results
 
# If no window centers found, just display orginal road image
else:
    output = np.array(cv2.merge((warped,warped,warped)),np.uint8)

# Display the final results
plt.imshow(output)
plt.title('window fitting results')
plt.show()


In [ ]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None
